In [267]:
!pip install plotly

    100% |████████████████████████████████| 1.1MB 656kB/s eta 0:00:01
  Running setup.py bdist_wheel for plotly ... done
  Stored in directory: /Users/Kevin/Library/Caches/pip/wheels/cc/87/3f/6a282eb21da5d8223472bed40ee023cdcf2e9691b117969a4c
Successfully built plotly


In [274]:
% matplotlib inline

In [277]:
import pandas as pd
import tensorflow as tf

import os, sys, re, json, time
import itertools
import collections
from IPython.display import display

# NumPy and SciPy for matrix ops
import numpy as np
import scipy.sparse

# Pandas because pandas are awesome
import pandas as pd
# Set pandas floating point display
pd.set_option('float_format', lambda f: "{0:.04f}".format(f))

# NLTK for NLP utils
import nltk

# Helper libraries
import utils

# Plotly imports.
import plotly.offline as plotly
plotly.offline.init_notebook_mode()
import plotly.graph_objs as go



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [2]:
!ls

2017-fall-main  Untitled.ipynb  Untitled1.ipynb reviews.csv


In [5]:
reviews = pd.read_csv("reviews.csv")

In [186]:
twitter = pd.read_csv("twitter.csv",header=5)

In [190]:
twitter.columns

Index(['Source', 'Host', 'Link', 'Time(ET)', 'Authority', 'Age', 'Gender',
       'Country', 'Location', 'Tags', 'Tags.1', 'Tags.2', 'Tags.3', 'Tags.4',
       'Tags.5', 'Tags.6', 'Star', 'Assigned', 'Sentiment', 'Alexa Rank',
       'Alexa Reach', 'Title', 'Snippet', 'Content', 'Unique ID', 'Language',
       'Followers', 'Following', 'Summary', 'Viewed', 'Commented',
       'Favourited', 'Rated', 'ImageUrl'],
      dtype='object')

In [149]:
reviews[pd.isnull(reviews.Text)==False].shape

(17766, 20)

In [12]:
reviews.head()

,Month/Year,Submission Day,# of Reviews,ASIN,Category,Product,YTD Revenue,YTD Units,LTD Star Rating,LTD Total # of Reviews,Replen Code,Monthly Star Rating,Verified?,Review Title,Text,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,Jan-17,03-JAN-17,1.0,B00SG7M6JW,Latch Bottles,"Latch BPA-Free Baby Bottle, 4 Ounce, 3 Pack",NaN,NaN,4.28,38.0,Planned Replenishment (PR),5,Y,NaN,I bought these as a supplement to our Tommee T...,NaN,NaN,NaN
1,Jan-17,02-JAN-17,1.0,B006SFUDQM,Bath,"Bubble Spout Guard, Blue",NaN,NaN,3.03,192.0,Planned Replenishment (PR),3,Y,NaN,This is a hard spout guard so I don't imagine ...,NaN,NaN,NaN
2,Jan-17,02-JAN-17,1.0,B00J40DY0A,Latch Bottles,"LATCH Transition Cup, 4 Ounce",NaN,NaN,4.05,592.0,Planned Replenishment (PR),5,Y,NaN,Perfect,NaN,NaN,NaN
3,Jan-17,01-JAN-17,1.0,B00PHWMKVS,Bath,"Wind Up Swimming Penguin Bath Toy, Pink",NaN,NaN,4.15,529.0,Planned Replenishment (PR),5,Y,NaN,This is adorable! My daughter loves it. The wa...,NaN,NaN,NaN
4,Jan-17,02-JAN-17,1.0,B009R27JVU,Diapering,"Arm & Hammer Diaper Pail Snap, Seal and Toss R...",NaN,NaN,4.09,397.0,Planned Replenishment (PR),5,N,NaN,Work well with my arm and hammer pail. The sce...,NaN,NaN,NaN


In [9]:
import datetime

In [11]:
datetime.datetime.strptime("03-JAN-17", '%d-%b-%y')

datetime.datetime(2017, 1, 3, 0, 0)

In [15]:
def to_date(x):
    if pd.isnull(x) == False:
        return datetime.datetime.strptime(x, '%d-%b-%y')

In [16]:
reviews["date"] = reviews["Submission Day"].apply(to_date)

In [19]:
reviews["rating"] = reviews['Monthly Star Rating']

In [92]:
reviews[['date','Product','Category','rating','Text']].head()

,date,Product,Category,rating,Text
0,2017-01-03,"Latch BPA-Free Baby Bottle, 4 Ounce, 3 Pack",Latch Bottles,5,I bought these as a supplement to our Tommee T...
1,2017-01-02,"Bubble Spout Guard, Blue",Bath,3,This is a hard spout guard so I don't imagine ...
2,2017-01-02,"LATCH Transition Cup, 4 Ounce",Latch Bottles,5,Perfect
3,2017-01-01,"Wind Up Swimming Penguin Bath Toy, Pink",Bath,5,This is adorable! My daughter loves it. The wa...
4,2017-01-02,"Arm & Hammer Diaper Pail Snap, Seal and Toss R...",Diapering,5,Work well with my arm and hammer pail. The sce...


In [39]:
import numpy as np
import re

## Later take a look at similarities by specific star rating
* Decided to look at full corpus for now

In [150]:
one_star_text = ''

for text in reviews.Text[reviews.rating == 1]:
    one_star_text += '<s>' + re.sub(r'[^\w\s]',' ',re.sub("'",'',text.lower()))
    how_many += 1
    
    

In [166]:
re.sub(' +',' ','The     quick brown    fox')

'The quick brown fox'

In [202]:
twitter_text = ''

for text in twitter.Content:
    twitter_text += ' <s> ' + re.sub(r'[^\w\s]',' ',re.sub("'",'',str(text).lower()))

In [204]:
twitter_text = re.sub(' +',' ',twitter_text)

In [205]:
twitter_text[:300]

' <s> join me mon nov 20 at humbercinemas for a movies4mommies screening of murder on the orient express doors open at 12 30pm screening is at 1pm enter for a chance to win prizes from munchkin_inc dkcanada plus get a clubcard so your 7th movie is free moviesformommies https t co zebaqewm81\n <s> plea'

In [242]:
amazon_text = ''
amazon_text_list = []

#how_many =0

for text in reviews.Text:
    amazon_text += ' <s> ' + re.sub(r'[^\w\s]',' ',re.sub("'",'',str(text).lower()))+ ' <\s> '
    #how_many+=1
    
for text in reviews.Text:
    amazon_text_list.append(' <s> ' + re.sub(r'[^\w\s]',' ',re.sub("'",'',str(text).lower()))+ ' <\s> ')   


In [233]:
amazon_text = re.sub(' +',' ',all_text)
amazon_text[:500]

' <s> i bought these as a supplement to our tommee tippee closer to nature bottles and i think i like these more unlike tommee tippee where the venting happens through the nipple the venting happens on the bottom so theres no risk of milk dripping out and leaking and making a mess when you breastfeed every drop counts so thats a huge plus ive tested these quite vigorously and i cant get the vents to leak at all even when submerged so thats a win the slow flow nipples that come with i <s> this is '

In [234]:
amazon_words = amazon_text.split(" ")

In [218]:
from collections import defaultdict

In [219]:
word_count = defaultdict(int)



for word in amazon_words:
    word_count[word]+=1
    

    

In [220]:
wc_df =pd.DataFrame.from_dict(word_count,orient="index")

In [221]:
#wc_df['word'] = wc_df.index

In [222]:
wc_df.reset_index(inplace=True)
wc_df.columns = ["word","count"]
wc_df['index_1'] = wc_df.index

In [248]:
wc_df.sort_values(by=["count","word"],ascending=False,inplace=True)

In [250]:
wc_df.tail()

,word,count,index_1
6653,0tool,1,6653
10793,07,1,10793
12963,02,1,12963
5690,01,1,5690
7619,00pm,1,7619


In [245]:
word_id = {}

for i,row in wc_df.iterrows():
    word_id[row.word]=i

In [247]:
word_id['to']

8

In [239]:
def cooccurrence_matrix(token_ids, V, K=2):
    # We'll use this as an "accumulator" matrix
    C = scipy.sparse.csc_matrix((V,V), dtype=np.float32)

    for k in range(1, K+1):
        print (u"Counting pairs (i, i \u00B1 %d) ..." % k)
        i = token_ids[:-k]  # current word
        j = token_ids[k:]   # k words ahead
        data = (np.ones_like(i), (i,j))  # values, indices
        Ck_plus = scipy.sparse.coo_matrix(data, shape=C.shape, dtype=np.float32)
        Ck_plus = scipy.sparse.csc_matrix(Ck_plus)
        Ck_minus = Ck_plus.T  # Consider k words behind
        C += Ck_plus + Ck_minus

    print ("Co-occurrence matrix: %d words x %d words" % (C.shape))
    print ("  %.02g nonzero elements" % (C.nnz))
    return C

In [244]:
amazon_words[:10]

['', '<s>', 'i', 'bought', 'these', 'as', 'a', 'supplement', 'to', 'our']

In [263]:
# Show co-occurrence on a toy corpus


amazon_text_list

#words


amazon_words

# ?

# sentence_to_ids adds "<s>" and "</s>"

#gives id of words



amazon_words_ids = [word_id[s] for s in amazon_words]

# Here's the important part


amazon_C = cooccurrence_matrix(amazon_words_ids, len(amazon_words), K = 1)

#ORdered from most common to least common than reverse alphabetical

#pretty_print_matrix(amazon_C.toarray(), rows=wc_df.word, 
#                          cols=wc_df.word, dtype=int, highlight="> 0")

Counting pairs (i, i ± 1) ...
Co-occurrence matrix: 663684 words x 663684 words
  3.1e+05 nonzero elements


In [264]:
def PPMI(C):
    """Tranform a counts matrix to PPMI.
    
    Args:
      C: scipy.sparse.csc_matrix of counts C_ij
    
    Returns:
      (scipy.sparse.csc_matrix) PPMI(C) as defined above
    """
    Z = float(C.sum())  # total counts
    # sum each column (along rows)
    Zc = np.array(C.sum(axis=0), dtype=np.float64).flatten()
    # sum each row (along columns)
    Zr = np.array(C.sum(axis=1), dtype=np.float64).flatten()
    
    # Get indices of relevant elements
    ii, jj = C.nonzero()  # row, column indices
    Cij = np.array(C[ii,jj], dtype=np.float64).flatten()
    
    ##
    # PMI equation
    pmi = np.log(Cij * Z / (Zr[ii] * Zc[jj]))
    ##
    # Truncate to positive only
    ppmi = np.maximum(0, pmi)  # take positive only
    
    # Re-format as sparse matrix
    ret = scipy.sparse.csc_matrix((ppmi, (ii,jj)), shape=C.shape,
                                  dtype=np.float64)
    ret.eliminate_zeros()  # remove zeros
    return ret

In [265]:
from sklearn.decomposition import TruncatedSVD
def SVD(X, d=100):
    """Returns word vectors from SVD.
    
    Args:
      X: m x n matrix
      d: word vector dimension
      
    Returns:
      Wv : m x d matrix, each row is a word vector.
    """
    transformer = TruncatedSVD(n_components=d, random_state=1)
    Wv = transformer.fit_transform(X)
    # Normalize to unit length
    Wv = Wv / np.linalg.norm(Wv, axis=1).reshape([-1,1])
    return Wv, transformer.explained_variance_

In [266]:
K = 1
d = 100
t0 = time.time()
C = cooccurrence_matrix(amazon_words_ids, len(amazon_words), K=K)

C_ppmi = PPMI(C)

Wv, _ = SVD(C_ppmi, d=d)


Counting pairs (i, i ± 1) ...
Co-occurrence matrix: 663684 words x 663684 words
  3.1e+05 nonzero elements


/Users/Kevin/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


In [280]:
Wv[:n,0]

array([ 0.12928815,  0.20913969,  0.29512393,  0.39446309,  0.4621689 ,
        0.35986042,  0.23747927,  0.17614105,  0.24344366,  0.24342609,
        0.27039268,  0.26038224,  0.24119994,  0.30443811,  0.4061299 ,
        0.30650422,  0.4149648 ,  0.4373221 ,  0.27008291,  0.4785889 ,
        0.48112256,  0.29809169,  0.24405147,  0.43308272,  0.40293112,
        0.44910094,  0.23946081,  0.52449255,  0.29548395,  0.57328559,
        0.19898579,  0.29338682,  0.23094789,  0.40806197,  0.26407156,
        0.29529095,  0.45716239,  0.53734463,  0.50078031,  0.32364343,
        0.43517266,  0.36636284,  0.451228  ,  0.5412659 ,  0.1334449 ,
        0.55821397,  0.38896546,  0.58745337,  0.38671899,  0.36133373,
        0.45322156,  0.28476233,  0.36507753,  0.460336  ,  0.23182231,
        0.46738567,  0.20564175,  0.45757927,  0.53873502,  0.20739648,
        0.46178911,  0.34243933,  0.48616449,  0.49800675,  0.25547489,
        0.4618298 ,  0.22716772,  0.36161231,  0.28083728,  0.46

In [286]:
n = 1000
data = [go.Scatter(x=Wv[:n,0], y=Wv[:n,1], text=amazon_words[:n],
                   mode='markers', textposition='bottom', hoverinfo='text')]
fig = go.Figure(data=data, layout=go.Layout(title="Word Embeddings", hovermode='closest'))
plotly.iplot(fig)